In [1]:
import numpy as np
from scipy import stats
import astropy
from astropy.table import Table
from astropy.table import Column
from astropy.nddata import CCDData
from astropy.nddata.utils import Cutout2D
from astropy import units as u
from astropy import wcs
import ccdproc
from ccdproc import trim_image
import PIL
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import glob
import os

# Creating a combined MSI catalogue

### It will be useful to create a single source catalogue with lon and lat of every MSI across the whole survey

In [2]:
file_location = os.path.join(r'C:\Users\clent\Documents\4th Year Project Code\MSIs', '*.vot')
filenames = glob.glob(file_location)
allvots = []
for i in filenames:
    allvots.append(Table.read(i))

In [3]:
count = np.arange(len(allvots))

which_frame = []
for i in count:
    which_frame.append(filenames[i].split('_')[0][52:])
    
print(which_frame)

['G002.5+0.0IFx', 'G005.5+0.0IFx', 'G008.5+0.0IFx', 'G011.5+0.0IFx', 'G014.5+0.0IFx', 'G017.5+0.0IFx', 'G020.5+0.0IFx', 'G023.5+0.0IFx', 'G026.5+0.0IFx', 'G029.5+0.0IFx', 'G032.5+0.0IFx', 'G035.5+0.0IFx', 'G038.5+0.0IFx', 'G041.5+0.0IFx', 'G044.5+0.0IFx', 'G047.5+0.0IFx', 'G050.5+0.0IFx', 'G053.5+0.0IFx', 'G056.5+0.0IFx', 'G059.5+0.0IFx', 'G249.5-0.5IFx', 'G252.5-0.5IFx', 'G255.5-0.5IFx', 'G258.5-0.5IFx', 'G261.5-0.5IFx', 'G264.5-0.5IFx', 'G267.5-0.5IFx', 'G273.5-0.5IFx', 'G276.5-0.5IFx', 'G279.5-0.5IFx', 'G282.5-0.5IFx', 'G285.5-0.5IFx', 'G288.5-0.5IFx', 'G291.5-0.5IFx', 'G294.5-0.5IFx', 'G297.5-0.5IFx', 'G300.5+000IFx', 'G300.5-0.5IFx', 'G303.5+000IFx', 'G306.5+000IFx', 'G309.5+000IFx', 'G312.5+000IFx', 'G315.5+000IFx', 'G318.5+000IFx', 'G321.5+000IFx', 'G321.5+000I', 'G324.5+000I', 'G327.5+000I', 'G330.5+000I', 'G333.5+000I', 'G336.5+000I', 'G339.5+000I', 'G342.5+000I', 'G345.5+000I', 'G348.5+000I', 'G351.5+000I', 'G354.5+000I', 'G357.5+000I']


In [4]:
for i in count:
    allvots[i].add_column(which_frame[i], name='Frame_name', index=0)
    
print(allvots)

[<Table length=229>
  Frame_name  island        lon                 lat        
    str13     int64       float64             float64      
------------- ------ ------------------ -------------------
G002.5+0.0IFx     18 3.5675960278050955 -1.5310183847924645
G002.5+0.0IFx     28 3.1940378574336203 -1.5141307930373298
G002.5+0.0IFx     35 2.7601788885529066 -1.4998802897603078
G002.5+0.0IFx     48 2.3324519374024573 -1.4748348986442494
G002.5+0.0IFx     57  2.480222135102502 -1.4735301912038097
G002.5+0.0IFx     58 2.2503284022303953  -1.472111896404913
G002.5+0.0IFx     59 2.7922194015915167 -1.4671410399044449
G002.5+0.0IFx     69 2.5267363792560493 -1.4514617007894017
G002.5+0.0IFx     88 3.8999195465726055 -1.4222916373064556
G002.5+0.0IFx    175  2.149650881316166 -1.3387105014587592
          ...    ...                ...                 ...
G002.5+0.0IFx   4355 2.2746404022101516   1.414503627031174
G002.5+0.0IFx   4373 3.5680578846447326  1.4327508491663028
G002.5+0.0IFx   4398

G357.5+000I   4723 357.04065909268354  1.5302758891078148]


In [5]:
MSIs = astropy.table.vstack(allvots)
print(MSIs)

  Frame_name  island        lon                 lat        
------------- ------ ------------------ -------------------
G002.5+0.0IFx     18 3.5675960278050955 -1.5310183847924645
G002.5+0.0IFx     28 3.1940378574336203 -1.5141307930373298
G002.5+0.0IFx     35 2.7601788885529066 -1.4998802897603078
G002.5+0.0IFx     48 2.3324519374024573 -1.4748348986442494
G002.5+0.0IFx     57  2.480222135102502 -1.4735301912038097
G002.5+0.0IFx     58 2.2503284022303953  -1.472111896404913
G002.5+0.0IFx     59 2.7922194015915167 -1.4671410399044449
G002.5+0.0IFx     69 2.5267363792560493 -1.4514617007894017
G002.5+0.0IFx     88 3.8999195465726055 -1.4222916373064556
G002.5+0.0IFx    175  2.149650881316166 -1.3387105014587592
          ...    ...                ...                 ...
  G357.5+000I   4532  357.2597086332006  1.2012389101837326
  G357.5+000I   4556 357.29974723983827  1.2325586242691844
  G357.5+000I   4576 357.31551255812525  1.2506173496848674
  G357.5+000I   4645 356.46124474455684 

In [6]:
MSIs.sort('lon')
print(MSIs)

  Frame_name  island        lon                  lat         
------------- ------ ------------------ ---------------------
G002.5+0.0IFx   2543  1.315452074188857    0.3883327858272292
G002.5+0.0IFx   2271 1.3282130307094773   0.15171515388251258
G002.5+0.0IFx   2210  1.331387557662496   0.08540959557617867
G002.5+0.0IFx   2207 1.3918056434096595   0.07994350294988505
G002.5+0.0IFx   2960 1.4025373781955321    0.6842554048985261
G002.5+0.0IFx   1813  1.412827078039409  -0.29274504922448485
G002.5+0.0IFx   1050 1.4519364732167512   -0.7849233480296052
G002.5+0.0IFx   2367 1.4621132532105157   0.23339107396579956
G002.5+0.0IFx   2674 1.5001846531693346    0.5021795128875668
G002.5+0.0IFx   2212 1.5063662881393067   0.08719897427540121
          ...    ...                ...                   ...
  G357.5+000I   2709 358.48405142813544    0.0681427979144989
  G357.5+000I   2801  358.4914599983926   0.12671449399809764
  G357.5+000I   2772  358.4994110812277   0.10757017051635603
  G357.5

In [7]:
#MeerACS: MeerKAT Aegean Complex Survey

IDs = np.arange(len(MSIs)) + 1
count = np.arange(len(MSIs))
MeerACS = []

for i in count:
    MeerACS.append('MeerACS' + str(IDs[i]))
    
print(MeerACS)

['MeerACS1', 'MeerACS2', 'MeerACS3', 'MeerACS4', 'MeerACS5', 'MeerACS6', 'MeerACS7', 'MeerACS8', 'MeerACS9', 'MeerACS10', 'MeerACS11', 'MeerACS12', 'MeerACS13', 'MeerACS14', 'MeerACS15', 'MeerACS16', 'MeerACS17', 'MeerACS18', 'MeerACS19', 'MeerACS20', 'MeerACS21', 'MeerACS22', 'MeerACS23', 'MeerACS24', 'MeerACS25', 'MeerACS26', 'MeerACS27', 'MeerACS28', 'MeerACS29', 'MeerACS30', 'MeerACS31', 'MeerACS32', 'MeerACS33', 'MeerACS34', 'MeerACS35', 'MeerACS36', 'MeerACS37', 'MeerACS38', 'MeerACS39', 'MeerACS40', 'MeerACS41', 'MeerACS42', 'MeerACS43', 'MeerACS44', 'MeerACS45', 'MeerACS46', 'MeerACS47', 'MeerACS48', 'MeerACS49', 'MeerACS50', 'MeerACS51', 'MeerACS52', 'MeerACS53', 'MeerACS54', 'MeerACS55', 'MeerACS56', 'MeerACS57', 'MeerACS58', 'MeerACS59', 'MeerACS60', 'MeerACS61', 'MeerACS62', 'MeerACS63', 'MeerACS64', 'MeerACS65', 'MeerACS66', 'MeerACS67', 'MeerACS68', 'MeerACS69', 'MeerACS70', 'MeerACS71', 'MeerACS72', 'MeerACS73', 'MeerACS74', 'MeerACS75', 'MeerACS76', 'MeerACS77', 'MeerAC

In [8]:
MSIs['island'] = MeerACS

In [9]:
MSIs.rename_column('island', 'Cat_ID')

In [10]:
print(MSIs)

  Frame_name     Cat_ID           lon                  lat         
------------- ------------ ------------------ ---------------------
G002.5+0.0IFx     MeerACS1  1.315452074188857    0.3883327858272292
G002.5+0.0IFx     MeerACS2 1.3282130307094773   0.15171515388251258
G002.5+0.0IFx     MeerACS3  1.331387557662496   0.08540959557617867
G002.5+0.0IFx     MeerACS4 1.3918056434096595   0.07994350294988505
G002.5+0.0IFx     MeerACS5 1.4025373781955321    0.6842554048985261
G002.5+0.0IFx     MeerACS6  1.412827078039409  -0.29274504922448485
G002.5+0.0IFx     MeerACS7 1.4519364732167512   -0.7849233480296052
G002.5+0.0IFx     MeerACS8 1.4621132532105157   0.23339107396579956
G002.5+0.0IFx     MeerACS9 1.5001846531693346    0.5021795128875668
G002.5+0.0IFx    MeerACS10 1.5063662881393067   0.08719897427540121
          ...          ...                ...                   ...
  G357.5+000I MeerACS19314 358.48405142813544    0.0681427979144989
  G357.5+000I MeerACS19315  358.4914599983926   

In [11]:
MSIs.write(r'C:\Users\clent\Documents\4th Year Project Code\overall_MSI_source_table.vot', overwrite=True, format='votable')

## To summarise, here I have imported the MSI source tables for all 58 frames, stacked them all to create one master source table, sorted by longitude (0 to 360), and created a unique ID system for future reference